# Embeddings

In [ ]:
import os, sys

import numpy as np
import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from metrics import Metrics
from data_processing import DataProcessing
from feature_extraction import SpacyFeatureExtraction, TfidfFeatureExtraction, SentenceTransformerFeatureExtraction

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Read csv files and load as df

In [3]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
predictions_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/pr

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.",1,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan forecasts that the revenue at Microsoft potentially decrease in Q3 of 2027.,1,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [4]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_1-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_1-observation/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_2-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/notebook_experiments/../data/observation_logs/batch_2-observation/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs

,Base Sentence,Sentence Label,Domain,Model Name,API Name,Batch ID,Template Number
0,The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.,0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,1
1,"On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,2
2,"A young investor predicts on 2025-03-15, the S&P 500 index may rise.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,3
3,"According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,4
4,"In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,5
5,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",0,finance,llama-3.1-70b-instruct,NAVI_GATOR,0,6
6,JPMorgan observed that the net profit at Microsoft had risen in September 2023.,0,finance,llama-3.3-70b-instruct,NAVI_GATOR,0,1


In [5]:
col_name = 'Base Sentence'
predictions = DataProcessing.df_to_list(predictions_df, col_name)
observations = DataProcessing.df_to_list(observations_df, col_name)
len(predictions), len(observations)

(694, 1891)

## Extract Embeddings with Spacy

In [6]:
by_prediction_df = pd.read_csv('../data/entailment/entailment-v1.csv')  
by_prediction_df

,Prediction,Observation,Entailment Label
0,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","In Q2 2025, a financial research advisor envisions that the research and development expenses at Google has some probability to remain stable.",NEUTRAL
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to the financial top executive at BlackRock, the stock price at Amazon may rise in Q4 2028.",NEUTRAL
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"In the fourth quarter of 2027, Wells Fargo envisions that the operating cash flow at Intel has some probability to remain stable.",NEUTRAL
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On March 1, 2029, the financial advisor at Wells Fargo envisions that the inflation rate at the Federal Reserve has some probability to remain stable.",NEUTRAL
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",NEUTRAL
6,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",NEUTRAL
7,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",NEUTRAL
8,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Apple stock price decreased in August 2024, according to Roger.",NEUTRAL
9,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL


In [7]:
by_prediction_df

,Prediction,Observation,Entailment Label
0,"Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.","In Q2 2025, a financial research advisor envisions that the research and development expenses at Google has some probability to remain stable.",NEUTRAL
1,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"According to the financial top executive at BlackRock, the stock price at Amazon may rise in Q4 2028.",NEUTRAL
2,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"In the fourth quarter of 2027, Wells Fargo envisions that the operating cash flow at Intel has some probability to remain stable.",NEUTRAL
3,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL
4,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On March 1, 2029, the financial advisor at Wells Fargo envisions that the inflation rate at the Federal Reserve has some probability to remain stable.",NEUTRAL
5,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"On November 10, 2022, to November 10, 2023, Citigroup speculates the research and development expenses at Amazon will likely increase.",NEUTRAL
6,JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.,"The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert at JPMorgan Chase.",NEUTRAL
7,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.",NEUTRAL
8,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Apple stock price decreased in August 2024, according to Roger.",NEUTRAL
9,"On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.","Intel stock price should stay the same in January 2028, according to a financial expert at Harvard University.",NEUTRAL


In [ ]:
disable_components = [""]
# pred_spacy_fe = SpacyFeatureExtraction(predictions_df, "Base Sentence")
pred_spacy_fe = SpacyFeatureExtraction(by_prediction_df, 'Prediction')
pred_sentence_features = pred_spacy_fe.sentence_feature_extraction()

obser_spacy_fe = SpacyFeatureExtraction(by_prediction_df, 'Observation')
obser_sentence_features = obser_spacy_fe.sentence_feature_extraction()

spacy_cs_metrics = Metrics.get_cosine_similarity(pred_sentence_features, obser_sentence_features)
by_prediction_df['Spacy'] = spacy_cs_metrics
by_prediction_df

100%|██████████| 12/12 [00:00<00:00, 297.20it/s]


## Extract Embeddings with Sentence Transformer

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

sentences = [] 
sentences.append(predictions[0])
sentences.append(observations[0])
print(sentences)

embeddings = model.encode(sentences)
print(embeddings.shape)
# => (3, 384)


# spacy_pred_sent_embedding.shape

similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

sent_transformer_pred_sent_embedding = model.encode(predictions[0])
sent_transformer_obser_sent_embedding = model.encode(observations[0])
similarities = model.similarity(sent_transformer_pred_sent_embedding, sent_transformer_obser_sent_embedding)
print(similarities)


In [ ]:
disable_components = [""]
# pred_spacy_fe = SpacyFeatureExtraction(predictions_df, "Base Sentence")
pred_st_fe = SentenceTransformerFeatureExtraction(by_prediction_df, 'Prediction')
st_pred_sentence_features = pred_st_fe.sentence_feature_extraction()

obser_spacy_fe = SentenceTransformerFeatureExtraction(by_prediction_df, 'Observation')
st_obser_sentence_features = obser_spacy_fe.sentence_feature_extraction()

st_cs_metrics = Metrics.get_cosine_similarity(st_pred_sentence_features, st_obser_sentence_features)
by_prediction_df['Sentence Transformer'] = st_cs_metrics
by_prediction_df

## Extract Embeddings with Bert 

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

from transformers import BertTokenizer, BertModel
def get_bert_embeddings(sentence: str):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained("bert-base-uncased")
    # text = "Replace me by any text you'd like."
    encoded_input = tokenizer(sentence, return_tensors='pt')
    # Get hidden states from BERT
    with torch.no_grad():
        output = model(**encoded_input)

    # Extract embeddings for [CLS] token
    sentence_embedding = output.last_hidden_state[:, 0, :].squeeze()

    return sentence_embedding

In [ ]:
bert_pred_sent_embedding = get_bert_embeddings(predictions[0])
bert_obser_sent_embedding = get_bert_embeddings(observations[0])

## Extract Embeddings with TF x IDF

In [ ]:
pred_tfidf_fe = TfidfFeatureExtraction(predictions_df, 'Base Sentence')
obser_tfidf_fe = TfidfFeatureExtraction(observations_df, 'Base Sentence')

pred_tfidf_df = pred_tfidf_fe.feature_scores(max_features=300)
obser_tfidf_df = obser_tfidf_fe.feature_scores(max_features=300)

In [ ]:
pred_tfidf_df[:1]

In [ ]:
tfidf_pred_embedding = pred_tfidf_df.iloc[:1 , 2:].to_numpy()

In [ ]:
obser_tfidf_df[:1]

In [ ]:
tfidf_obser_embedding = obser_tfidf_df.iloc[:1 , 2:].to_numpy()
tfidf_obser_embedding

## Extract

In [ ]:
# roberta_fe = RobertaFeatureExtraction(by_prediction_df, type_of_df="Pivot Table")
# predicton_embeddings, observation_embeddings = roberta_fe.entailment_feature_extraction()

## Embeddings Similary

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_cosine_similarity(prediction_embeddings: np.array, observation_embeddings: np.array):

    # make them (1 × vector_dim) for sklearn
    pred_sent_embedding_reshaped = prediction_embeddings.reshape(1, -1)
    obser_sent_embedding_reshaped = observation_embeddings.reshape(1, -1)

    sim = cosine_similarity(pred_sent_embedding_reshaped, obser_sent_embedding_reshaped)[0, 0]
    
    return sim

In [ ]:
spacy_cs_metric = get_cosine_similarity(spacy_pred_sent_embedding, spacy_obser_sent_embedding)
# sent_tranformer_cs_metric = get_cosine_similarity(sent_transformer_pred_sent_embedding, sent_transformer_obser_sent_embedding)
# bert_cs_metric = get_cosine_similarity(bert_pred_sent_embedding, bert_obser_sent_embedding)
# tfidf_cs_metric = get_cosine_similarity(tfidf_pred_embedding, tfidf_obser_embedding)

# spacy_cs_metric, sent_tranformer_cs_metric, bert_cs_metric, tfidf_cs_metric
spacy_cs_metric